In [181]:
## Getting content from tavily APIs

user_query = "What are the effects of parental involvement on academic performance?"
user_query = "How can a father and mother influence a child's performance and grades in school?"

from tavily import TavilyClient
import shutil
import os

tavily = TavilyClient(api_key=os.getenv('TAVILY_API_KEY'))

response = tavily.search(query=user_query, 
                         search_depth="advanced", 
                         include_raw_content=True,
                         max_results=5)

# response = tavily.get_search_context(query=user_query, search_depth="advanced", max_tokens=1500)
# response2 = tavily.qna_search(query=user_query)

In [182]:
response

{'query': "How can a father and mother influence a child's performance and grades in school?",
 'follow_up_questions': None,
 'answer': None,
 'images': None,
 'results': [{'title': 'Support from Mothers and Fathers on Academic Functioning: More ...',
   'url': 'https://link.springer.com/article/10.1007/s10826-023-02590-4',
   'content': 'Results highlighted risk in early adolescence was associated with decreasing grades and attendance over the course of a school year, but found that support from mothers and fathers reduced the decline in grades, as well as attendance. Mother and father support were associated with end of the year grade point average (GPA) and father support was ...',
   'score': 0.9426,
   'raw_content': 'Advertisement\nSupport from Mothers and Fathers on Academic Functioning: More Similarities than Differences Across Socioeconomic Groups\n425 Accesses\n2 Citations\nExplore all metrics\nAbstract\nEarly adolescence is a time of unique risks in relation to academic outc

In [183]:
# if os.path.exists('tavily_data'):
#     shutil.rmtree('tavily_data')
#     os.makedirs('tavily_data')
# else:
#     os.makedirs('tavily_data')

meta_map = {}

for count, result in enumerate(response['results']):
    f = open("tavily_data/file{}.txt".format(count), "a", encoding="utf-8")
    meta_map[count] = {'title':result['title'], 'url':result['url']}
    f.truncate(0)
    f.write(result['raw_content'])
    f.close()

In [163]:
meta_map

{0: {'title': 'Support from Mothers and Fathers on Academic Functioning: More ...',
  'url': 'https://link.springer.com/article/10.1007/s10826-023-02590-4'},
 1: {'title': 'The relationship between parenting engagement and academic performance ...',
  'url': 'https://www.nature.com/articles/s41598-022-26258-z'}}

In [80]:
## querying AI model

from pathlib import Path
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
)
from llama_index.llms.openai import OpenAI
from typing import List, Optional
from llama_index.llms.huggingface import (
    HuggingFaceInferenceAPI,
    HuggingFaceLLM,
)
from pydantic import BaseModel
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import IndexNode
from llama_index.core.embeddings import resolve_embed_model
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.query_engine import RetrieverQueryEngine

In [150]:
from llama_index.core import Settings
# Settings.chunk_size = 2056
Settings.chunk_size = 512
Settings.llm = OpenAI(model="gpt-4", api_key=os.getenv('OPENAI_API_KEY'))

In [184]:
documents = SimpleDirectoryReader("tavily_data").load_data()
index = VectorStoreIndex.from_documents(documents)

In [152]:
# HF_TOKEN: Optional[str] = os.getenv("HUGGING_FACE_TOKEN")
gpt4 = OpenAI(model="gpt-4", api_key=os.getenv('OPENAI_API_KEY'))
query = user_query

In [185]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import IndexNode


node_parser = SentenceSplitter(chunk_size=256)

base_nodes = node_parser.get_nodes_from_documents(documents)
# set node ids to be a constant
for idx, node in enumerate(base_nodes):
    node.id_ = f"node-{idx}"

In [186]:
# base_retriever = index.as_retriever(similarity_top_k=5)
base_index = VectorStoreIndex(base_nodes)
base_retriever = base_index.as_retriever(similarity_top_k=5)
retrievals = base_retriever.retrieve(query)
ans = ''
for index, n in enumerate(retrievals):
    if 'https:/' not in n.text:
        file_name = n.metadata['file_name'][n.metadata['file_name'].rfind('/'):]
        curr_id = int(''.join(filter(str.isdigit, file_name)))
        # display_source_node(n, source_length=1500)
        ans += 'Reference {}: '.format(index) + n.text + '\n'
query_engine_base = RetrieverQueryEngine.from_args(base_retriever, llm=gpt4)
response = query_engine_base.query(query)

/file0.txt 0
/file0.txt 0
/file0.txt 0
/file0.txt 0
/file0.txt 0


In [187]:
print(ans)

Reference 0: Results highlighted risk in early adolescence was associated with decreasing grades and attendance over the course of a school year, but found that support from mothers and fathers reduced the decline in grades, as well as attendance. Mother and father support were associated with end of the year grade point average (GPA) and father support was associated with days absent. Finally, mother support moderated the association between financial hardship and GPA. Interventions to support the academic success of middle school students, and ways to facilitate positive school-family partnerships to promote supportive relationships are discussed.
Highlights
Our study investigated the role of mother and father support and family socio-economic hardship on academic outcomes over time with a diverse sample of middle school youth; findings indicated that early adolescence was associated with decreasing grades and attendance.
Support from both mothers and fathers was associated with high